In [ ]:
import numpy as np
from numpy import array
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def filter(csv, affectedCountry):
  df=pd.read_excel("/content/RiskScore (1).xlsx")
  df.set_index('Geo Country',inplace=True)
  q=df.loc[[affectedCountry]]['Incident_Count'][affectedCountry]
  res = q.strip('][').split(', ')
  integer_map = map(int, res)
  res = list(integer_map)
  a, b = res, res[len(res)-4:len(res)]
  def prediction(a, b):
    train=a
    test=b
    def split_sequence(sequence, n_steps):
      X, y = list(), list()
      for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
          break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
      return array(X), array(y)
    raw_seq = train
  # choose a number of time steps
    n_steps = 4
    # split into samples
    X, y = split_sequence(raw_seq, n_steps)
    # reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
    n_features = 1
    n_seq = 2
    n_steps = 2
    X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=500, verbose=0)
    x_input = test
    #x_input = x_input.reshape((1, n_seq, n_steps, n_features))
    ans=[]
    for i in range(0, 4):
      ip=x_input[len(x_input)-4: len(x_input)]
      temp=array(ip)
      ip = temp.reshape((1, n_seq, n_steps, n_features))
      yhat = model.predict(ip, verbose=0)
      x=yhat.tolist()
      ans.append(x[0][0])
      x_input.append(x[0][0])

      #x_input = x_input.reshape((1, n_seq, n_steps, n_features))
    finalOutput={'Prediction:':ans}
    return finalOutput['Prediction:']
  return prediction(a,b)


In [ ]:
upcomingCases=filter("/content/RiskScore (1).xlsx", "Myanmar")

In [ ]:

print(upcomingCases)

[10.452516555786133, 7.895261287689209, 11.176444053649902, 7.580087184906006]
